<br>

# Notebook 2 - Crop Simulation

<hr>
This key module simulates all the possible crop cycles to  find the best crop cycle that produces maximum yield for a particular grid. During the simulation process for each grid, 365 crop cycle simulations are performed. Each simulation corresponds to cycles that start from each day of the year (starting from Julian date 0 to Julian date 365). Similarly, this process is performed by the program for each grid in the study area. 

Prepared by Geoinformatics Center, AIT
<hr>


### Google drive connection
In this step, we will connect to Google Drive service and mount the drive where we will start our PyAEZ project

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [ ]:
# 'Installing neccessary packages'
# !pip install gdal

Now, we will import the specific Python packages we need for PyAEZ.

In [1]:
'''import supporting libraries'''
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

import rioxarray as rio

# requires environment with openpyxl

Setting the working directory -- where our PyAEZ project is located.

In [2]:
# 'Set the working directory'
# work_dir = r'D:\PyAEZv2.1_Draft'  # Please change this to your working directory
# os.chdir(work_dir)
# sys.path.append('./pyaez/')
# os.getcwd()

# These paths must be modified by each user to point to the appropriate data files and directories

# branch version tag
revname='v21pv'

# # Kerrie laptop
work_dir = r'C://Users/kerrie/Documents/01_LocalCode/repos/PyAEZ/pyaez2.1/pyaez2.1_parvec_module2/' # path to your PyAEZ repo
out_path = work_dir+'NB2outputs/' # path for saving output data
dir_toplev=r'C://Users/kerrie/Documents/01_LocalCode/'
# # china
# domain='china'
# data_dir = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/npy/' # path to your data
# maskfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/tif/mask.tif'# subset for no antarctica, 1800 lats
# elevfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/tif/elev.tif'
# soilfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/tif/soil_terrain_lulc_china_08333.tif'
# global
domain='global'
# data_dir = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global/npy/' # path to your data
# maskfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global/tif/mask_2268708_5m.tif'# subset for no antarctica, 1800 lats
# elevfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global/tif/elev_2268708_5m.tif'
# soilfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global/tif/soil_terrain_lulc_global_08333.tif'
# laos
domain='laos'
data_dir = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/' # path to your data
maskfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/Laos_mask.npy'# subset for no antarctica, 1800 lats
elevfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/lao_elevation.npy'
soilfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/Laos_soil.npy'
# maskfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/LAO_Admin.tif'# subset for no antarctica, 1800 lats
# elevfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/LAO_Elevation.tif'
# soilfile = r'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/laos/LAO_soil_terrain_lulc.tif'

# Check whether the specified path exists or not
isExist = os.path.exists(out_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(out_path)
   print("The new directory is created!")

daskpath=os.path.join(dir_toplev, "dask-worker-space-can-be-deleted")

isExist = os.path.exists(daskpath)
if not isExist:
   os.mkdir(daskpath)
   print("dask worker directory created")

mask_path=maskfile

Check and create data output folder

In [3]:
# import os
# folder_path = './data_output/NB2/'
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)
#     print("Folder created successfully.")
# else:
#     print("Folder already exists.")


<hr>

## MODULE 2: CROP SIMULATION
Now, we will start executing the routines in Module 2


First, we initiate Module 2 Class instance by invoking the following commands:

In [4]:
# '''importing library'''


# from pyaez import CropSimulation, UtilitiesCalc

# %load_ext autoreload
# %autoreload 2
# # Import Module 2 and initate Class intance

# aez = CropSimulation.CropSimulation()

# obj_util = UtilitiesCalc.UtilitiesCalc()


sys.path.append(work_dir)

import CropSimulation_2023OCT17 as CropSimulation
aez = CropSimulation.CropSimulation()

import UtilitiesCalc as UtilitiesCalc
obj_utilities=UtilitiesCalc.UtilitiesCalc()


# import ClimateRegime as ClimateRegime
# clim_reg = ClimateRegime.ClimateRegime()

### Importing the climate dataset and the geographical data/rasters.

The package expects six climate variables, as daily or monthly observations, as Numpy arrays.
Arrays must be 3-dimensional, with the third axes containing the time dimension.
Unit of measures are expected as follows:
- Minimum temperature = Degree Celsius
- Maximum temperature = Degree Celsius
- Precipitation = Accumulated mm / day (or per month)
- Solar radiation = W/m^2
- Wind speed = Average m/s
- Relative humidity = Average fraction (0 to 1)

In addition to climate data, the system requires:
- A binary admin_mask, with 0 and 1 values. 0 pixels values will be not executed, while 1 pixels values will be executed
- An elevation layer
- Soil/terrain/special land cover classes
  

**All the datasets must have the same shape.**


## Reading Data

In [5]:
daily = True          # Type of climate data. True: daily, False: monthly (NOTE: parallel processing not yet implemented for Daily=False, notebook may fail)
parallel=True #False# # flag for dask parallel processing (for speed). True: use dask. False: no dask, only numpy
mask_value = 0        # pixel value in admin_mask to exclude from the analysis

if parallel:
    import dask.array as da
    import dask

# # If parallel=True, load the inputs as lazy dask arrays, data type float32 
# max_temp = da.from_npy_stack(data_dir+'Tmax-2m365/').astype('float32')  # maximum temperature
# min_temp = da.from_npy_stack(data_dir+'Tmin-2m365/').astype('float32')  # minimum temperature
# precipitation = da.from_npy_stack(data_dir+'Precip365/').astype('float32')  # precipitation
# rel_humidity = da.from_npy_stack(data_dir+'Rhum365/').astype('float32')  # relative humidity
# wind_speed = da.from_npy_stack(data_dir+'Wind-2m365/').astype('float32') # wind speed measured at two meters
# short_rad = da.from_npy_stack(data_dir+'Srad365/').astype('float32')  # shortwave radiation
# mask=da.from_array(gdal.Open(maskfile).ReadAsArray())
# elevation=da.from_array(gdal.Open(elevfile).ReadAsArray())
# soil_terrain_lulc=da.from_array(gdal.Open(soilfile).ReadAsArray())

# # If parallel=False, comment out the above and instead load the inputs as numpy arrays, data type float32 
# max_temp = np.load(data_dir+'Tmax-2m365/0.npy').astype('float32')  # maximum temperature
# min_temp = np.load(data_dir+'Tmin-2m365/0.npy').astype('float32')  # minimum temperature
# precipitation = np.load(data_dir+'Precip365/0.npy').astype('float32')  # precipitation
# rel_humidity = np.load(data_dir+'Rhum365/0.npy').astype('float32')  # relative humidity
# wind_speed = np.load(data_dir+'Wind-2m365/0.npy').astype('float32') # wind speed measured at two meters
# short_rad = np.load(data_dir+'Srad365/0.npy').astype('float32')  # shortwave radiation
# mask=gdal.Open(maskfile).ReadAsArray()
# elevation=gdal.Open(elevfile).ReadAsArray()
# soil_terrain_lulc=gdal.Open(soilfile).ReadAsArray()

# FOR LAOS
# If parallel=True, load the inputs as lazy dask arrays, data type float32 
max_temp=  da.from_array(np.load(data_dir+'Laos_max_temp_2022.npy').astype('float32'))
min_temp = da.from_array(np.load(data_dir+'Laos_min_temp_2022.npy').astype('float32'))  # minimum temperature
precipitation = da.from_array(np.load(data_dir+'Laos_precipitation_2022.npy').astype('float32'))  # precipitation
rel_humidity = da.from_array(np.load(data_dir+'Laos_rel_humid_2022.npy').astype('float32'))  # relative humidity
wind_speed = da.from_array(np.load(data_dir+'Laos_wind_sp_2022.npy').astype('float32')) # wind speed measured at two meters
short_rad = da.from_array(np.load(data_dir+'Laos_solar_rad_2022.npy').astype('float32'))  # shortwave radiation
mask=da.from_array(np.load(maskfile).astype('float32'))
elevation=da.from_array(np.load(elevfile).astype('float32'))
soil_terrain_lulc=da.from_array(np.load(soilfile).astype('float32'))
# mask=da.from_array(gdal.Open(maskfile).ReadAsArray())
# elevation=da.from_array(gdal.Open(elevfile).ReadAsArray())
# soil_terrain_lulc=da.from_array(gdal.Open(soilfile).ReadAsArray())

mask.shape, max_temp.shape


((85, 74), (85, 74, 365))

This section contains parameters that can be modified by the user:
- lat_min = minimum latitude of analysis
- lat_max = maximum latitude of analysis
- mask_value = the value in the admin_mask to exclude from the analysis (typically 0)
- daily = whether climate input data are daily (True) or monthly (False)

In [6]:
# Define the Area-Of-Interest's geographical extents

# if lat_min/lat_max values defined below are located at pixel center --> set lat_centers to True 
# if they are located at the exterior pixel edge --> set lat_centers to False
lat_centers=True 

# provide min and max latitudes (either set manually or read from a data file)
lat_min = 13.87#18.04167
lat_max = 22.59#53.625
# I'm limiting precision here to avoid any misalignment of the grid cells
# lats=rio.open_rasterio(maskfile)['y'].data   # get array of latitudes from maskfile
# lat_min = np.trunc(lats.min()*100000)/100000 # min lat value at pixel center, limit precision to 5 decimal places
# lat_max = np.trunc(lats.max()*100000)/100000 # max lat value at pixel center, limit precision 5 decimal places

### Loading the imported data into the Object Class ('*aez*' Class)

In [8]:
aez.setParallel(max_temp,parallel)

##### THIS IS WHERE YOU LEFT OFF ########

 
# aez.setStudyAreaMask(mask, mask_value)
# aez.setLocationTerrainData(lat_min, lat_max, lat_centers, elevation)
# if daily:
#     aez.setDailyClimateData(
#         min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
# else:
#     aez.setMonthlyClimateData(
#         min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)


In [ ]:
print(aez.__dict__.keys())
print(len(aez.__dict__.keys()))

In [ ]:
for key in aez.__dict__.keys():
    dtype=type(aez.__dict__[key])
    if dtype==bool:
        print(key,'boolean')
    if dtype==tuple:
        print(key,'tuple')
    if dtype in [int,float]:
        print(key,'scalar')
    if dtype==np.ndarray:
        print(key,'np array')
    if dtype==dask.array.core.Array:
        print(key,'da array')
    if dtype not in [bool,tuple,int,float,np.ndarray,dask.array.core.Array]:
        print(key,'***',dtype,'***')

### Setting up the crop parameter/ crop cycle parameter and soil water parameters (Mandatory)

In [ ]:
# setting up the crop parameters, crop cycle and soil water parameters ***mandatory step
# New function, reading crop-specific biomass/yield loss/TSUM screening factors from excel sheet, xlsx file.
aez.readCropandCropCycleParameters(file_path = r'input_crop_TSUM_parameters_maiz_sugar.xlsx', 
                                   crop_name = 'sugarcane')


aez.setSoilWaterParameters(Sa= 100*np.ones((mask.shape)), pc=0.5)

### Setting up the thermal screening parameters (Optional)

In [ ]:
# If you're simulating perennial crops, this thermal screening is mandatory
# Compute Thermal Climate
tclimate = aez.getThermalClimate()

# Compute permafrost
permafrost_eval = aez.AirFrostIndexandPermafrostEvaluation()
frost_index = permafrost_eval[0]
permafrost_class = permafrost_eval[1]
# tclimate = gdal.Open("./data_output/NB1/LAO_ThermalClimate.tif").ReadAsArray()# User to change this TClimate file 
# permafrost_class = gdal.Open("./data_output/NB1/LAO_permafrost.tif").ReadAsArray()# User to change this permafrost file

# Thermal Climate screening
aez.setThermalClimateScreening(tclimate, no_t_climate=[2,6,7,8,9,10,11,12])

# New Thermal Screening: Permafrost Screening
aez.setPermafrostScreening(permafrost_class= permafrost_class)

# Updated Temperature Profile screenign routine
aez.setupTypeBConstraint(file_path = r'crop-specific_rule_maiz_sugar.xlsx', 
                         crop_name = 'sugarcane')

# Is the crop perennial? (True/False)
is_perennial = True

### Perennial Adjustment (Mandatory Step for Perennials)


In [ ]:
# Create climatic indicators independently in this notebook
lgpt5 = aez.getThermalLGP5()
lgpt10 = aez.getThermalLGP10()
lgp = aez.getLGP(Sa=100., D=1.) #has to be after LGPt are computed

# # Uncomment and change file path if you would like to load
# # the TIFF file saved from NB1
# lgp = gdal.Open("../pyaez2.1_parvec/NB1outputs/LGP_china_v21pv.tif").ReadAsArray() # length of growing period
# lgpt5 = gdal.Open("../pyaez2.1_parvec/NB1outputs/LGPt5_china_v21pv.tif").ReadAsArray()# temperature growing period 5 deg
# lgpt10 = gdal.Open("../pyaez2.1_parvec/NB1outputs/LGPt10_china_v21pv.tif").ReadAsArray() # temperature growing period 10 deg

aez.ImportLGPandLGPTforPerennial(lgp, lgpt5, lgpt10)

In [ ]:
print(aez.__dict__.keys())
print(len(aez.__dict__.keys()))
'pet_daily' in aez.__dict__.keys()

In [ ]:
aez.min_temp

### Simulate crop cycle


In [ ]:
'''run simulations'''
aez.simulateCropCycle( start_doy=1, end_doy=365, step_doy=1, leap_year=False) # results are in kg / hectare

In [ ]:
# from calendar import isleap
# isleap(2020)

In [ ]:
aez.cycle_len, aez.LAi,aez.aLAI,aez.bLAI
aez.permafrost_class
aez.pet_daily # dask array

In [ ]:
# CropSimulation.py
# create a mask that covers some conditions where we don't need to compute crop sims
# conditions are: under the mask, 
#                 permafrost categories 1&2, 
#                 user input thermal climate screening, 
#                 annual mean T less than min crop temp

# mask screening
multi_screen=aez.im_mask

# permafrost screening
multi_screen=np.where((aez.permafrost_class==1)|(aez.permafrost_class==2),0,multi_screen)

# thermal climate screening
for tclim_val in aez.no_t_climate:
    multi_screen=np.where(aez.t_climate==tclim_val,0,multi_screen)

# minimum temperature screening
multi_screen=np.where(np.round(aez.annual_Tmean,0)<aez.min_temp,0,multi_screen)

multi_screen3D=np.broadcast_to(multi_screen[:,:,np.newaxis],(aez.im_height,aez.im_width,aez.doy_end))
multi_screen.shape,multi_screen3D.shape

In [ ]:
npts=np.sum(multi_screen).compute()
print('npts=',npts)

fig = plt.figure(figsize=(12,3))
plt.imshow(multi_screen, cmap='viridis', vmin=0, vmax=1,interpolation='none')  
plt.colorbar(shrink=0.8)

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.LGP, cmap='viridis', vmin=0, vmax=365,interpolation='none')  
# plt.colorbar(shrink=0.8)


In [ ]:
# CropSimulation.py
"""No adjustment of cycle length, LAI and HI for non-perennials"""
if not aez.perennial:
    #  COME BACK TO THIS LATER
    pass
    # aez.cycle_len_rain = aez.cycle_len
    # aez.LAi_rain = aez.LAi
    # aez.HI_rain = aez.HI

    # aez.cycle_len_irr = aez.cycle_len
    # aez.LAi_irr = aez.LAi
    # aez.HI_irr = aez.HI

else:
    """Adjustment of cycle length, LAI and HI for Perennials"""
    aez.set_adjustment = True

    """ Adjustment for RAINFED conditions"""
    # LGP duration will be efficient cycle length for rainfed conditions
    # Later, we use LGP length to adjust for LAI and HI for rainfed conditions    
    aez.cycle_len_rain=np.where(aez.LGP<aez.cycle_len,aez.LGP,aez.cycle_len)
    aez.cycle_len_rain=np.where(multi_screen,aez.cycle_len_rain,np.float32(np.nan))

    # ELIMINATE adjustForPerennialCrop, DOUBLE CHECK THIS ISN'T CALLED ELSEWHERE
    # adjust for perrennial crops
    aez.LAi_rain = np.where(multi_screen,aez.LAi,np.float32(np.nan))
    LAi_rain_adjust = aez.LAi * ((aez.cycle_len_rain-aez.aLAI)/aez.bLAI)
    aez.LAi_rain = np.where(np.isfinite(aez.LAi_rain)&(aez.LGP<aez.cycle_len),LAi_rain_adjust,aez.LAi_rain)
    del LAi_rain_adjust

    aez.HI_rain = np.where(multi_screen,aez.HI,np.float32(np.nan))
    HI_rain_adjust = aez.HI * ((aez.cycle_len_rain-aez.aHI)/aez.bHI)
    aez.HI_rain = np.where(np.isfinite(aez.HI_rain)&(aez.LGP<aez.cycle_len),HI_rain_adjust,aez.HI_rain)
    del HI_rain_adjust

    """ Adjustment for IRRIGATED conditions"""
    aez.LAi_irr = np.where(multi_screen,aez.LAi,np.float32(np.nan))
    aez.HI_irr = np.where(multi_screen,aez.HI,np.float32(np.nan))      
    """Use LGPT5 for minimum temperature less than or equal to five deg Celsius"""
    if aez.min_temp <=5:
        aez.cycle_len_irr=np.where(aez.LGPT5<aez.cycle_len,aez.LGPT5,aez.cycle_len)
        aez.cycle_len_irr=np.where(multi_screen,aez.cycle_len_irr,np.float32(np.nan))

        # ELIMINATE adjustForPerennialCrop, DOUBLE CHECK THIS ISN'T CALLED ELSEWHERE
        # adjust for perrennial crops
        LAi_irr_adjust = aez.LAi * ((aez.cycle_len_irr-aez.aLAI)/aez.bLAI)
        aez.LAi_irr = np.where(np.isfinite(aez.LAi_irr)&(aez.LGPT5<aez.cycle_len),LAi_irr_adjust,aez.LAi_irr)
        del LAi_irr_adjust

        HI_irr_adjust = aez.HI * ((aez.cycle_len_irr-aez.aHI)/aez.bHI)
        aez.HI_irr = np.where(np.isfinite(aez.HI_irr)&(aez.LGPT5<aez.cycle_len),HI_irr_adjust,aez.HI_irr)
        del HI_irr_adjust   

    """Use LGPT10 for minimum temperature greater than five deg Celsius"""
    if aez.min_temp>5:
        aez.cycle_len_irr=np.where(aez.LGPT10<aez.cycle_len,aez.LGPT10,aez.cycle_len)
        aez.cycle_len_irr=np.where(multi_screen,aez.cycle_len_irr,np.float32(np.nan))

        # ELIMINATE adjustForPerennialCrop, DOUBLE CHECK THIS ISN'T CALLED ELSEWHERE
        # adjust for perrennial crops
        LAi_irr_adjust = aez.LAi * ((aez.cycle_len_irr-aez.aLAI)/aez.bLAI)
        aez.LAi_irr = np.where(np.isfinite(aez.LAi_irr)&(aez.LGPT10<aez.cycle_len),LAi_irr_adjust,aez.LAi_irr)
        del LAi_irr_adjust

        HI_irr_adjust = aez.HI * ((aez.cycle_len_irr-aez.aHI)/aez.bHI)
        aez.HI_irr = np.where(np.isfinite(aez.HI_irr)&(aez.LGPT10<aez.cycle_len),HI_irr_adjust,aez.HI_irr)
        del HI_irr_adjust 


     

# print(aez.LAi,aez.aLAI,aez.bLAI)
# print(aez.HI,aez.aHI,aez.bHI)

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.cycle_len_rain, cmap='viridis', vmin=0, vmax=366,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.cycle_len_rain')

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.cycle_len_irr, cmap='viridis', vmin=0, vmax=366,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.cycle_len_irr')


# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.LAi_rain, cmap='viridis', vmin=0, vmax=10,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.LAi_rain')

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.LAi_irr, cmap='viridis', vmin=0, vmax=10,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.LAi_irr')

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.HI_rain, cmap='viridis', vmin=0, vmax=.25,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.HI_rain')

# fig = plt.figure(figsize=(12,3))
# plt.imshow(aez.HI_irr, cmap='viridis', vmin=0, vmax=.25,interpolation='none')  
# plt.colorbar(shrink=0.8)
# plt.title('aez.HI_irr')


In [ ]:
# minT_screen=np.where(multi_screen3D,min_temp,np.float32(np.nan))
# maxT_screen=np.where(multi_screen3D,max_temp,np.float32(np.nan))

# minT_daily_2year = np.tile(minT_screen, 2).compute()
# maxT_daily_2year = np.tile(minT_screen, 2).compute()
# # shortRad_daily_2year = np.tile(shortRad_daily_point, 2)

# # totalPrec_daily_2year = np.tile(totalPrec_daily_point, 2)
# # pet_daily_2year = np.tile(pet_daily_point, 2)

In [ ]:
test=aez.cycle_len_rain.compute()
test_mask0=np.where(np.isfinite(test),1,0)
npts=np.sum(test_mask0)
print('npts=',npts)

test_mask=np.where(test>0,1,0)
npts=np.sum(test_mask)
print('npts=',npts)

fig = plt.figure(figsize=(12,3))
plt.imshow(test, cmap='viridis', vmin=0, vmax=365,interpolation='none')  
plt.colorbar(shrink=0.8)


In [ ]:
# len(np.unique
# (test)),np.unique(test)
# np.nan+1

In [ ]:
# unique_cycles=np.unique(aez.cycle_len_rain.compute())
# ncycle_len=len(unique_cycles)


# cycle_mask=np.where(aez.cycle_len_rain.compute()==100,1,0)

# fig = plt.figure(figsize=(12,3))
# plt.imshow(cycle_mask, cmap='viridis', vmin=0, vmax=1,interpolation='none')  
# plt.colorbar(shrink=0.8)


In [ ]:
# def calculateCycleQuantities(cycle_tmin,cycle_tmax):
#     meanT_daily=0.5*(cycle_tmin+cycle_tmax)
#     cycle_meanT=meanT_daily.mean(axis=2)
#     return cycle_meanT


# def calculateCycleQuantities(istart,group_c_len,cycle_len):
#     if group_c_len in unique_cycles:
#         iend=istart+int(group_c_len)
#         # find all pixels with this length of growing season
#         cycle_mask=np.where(cycle_len==group_c_len,1,0)
#         # expand dims
#         cycle_mask3D=np.broadcast_to(cycle_mask[:,:,np.newaxis],vardims)
#         # mask input variables in space
#         tmin_masked=np.where(cycle_mask3D,minT_daily_2year,np.float32(np.nan))
#         tmax_masked=np.where(cycle_mask3D,maxT_daily_2year,np.float32(np.nan))
#         # subset input variables in time
#         cycle_tmin=tmin_masked[:,:,istart:iend]
#         cycle_tmax=tmax_masked[:,:,istart:iend]

#         meanT_daily=0.5*(cycle_tmin+cycle_tmax)
#         cycle_meanT=meanT_daily.mean(axis=2)
#     else:
#         cycle_meanT=np.empty((aez.im_height,aez.im_width),dtype('float32'))
#         cycle_meanT[:]=np.float32(np.nan)
#     return cycle_meanT

# # def calculateCycleQuantities(istart,group_c_len,cycle_len,vardims,cycle_tmin,cycle_tmax):
min_lgp=30
def calculateCycleQuantities(istart,group_c_len,cycle_len,tmin,tmax):
    if (group_c_len in unique_cycles)&(group_c_len>min_lgp):
        iend=istart+int(group_c_len)
        # find all pixels with this length of growing season
        # cycle_mask=np.where(cycle_len==group_c_len,1,0)
        # expand dims
        # cycle_mask3D=np.broadcast_to(cycle_mask[:,:,np.newaxis],vardims)
        # mask input variables in space
        # print('func 1')
        # tmin_masked=np.where(cycle_mask3D,tmin,np.float32(np.nan))
        # tmax_masked=np.where(cycle_mask3D,tmax,np.float32(np.nan))
        # subset input variables in time
        # print('func 2')
        # cycle_tmin=tmin_masked[:,:,istart:iend]
        # cycle_tmax=tmax_masked[:,:,istart:iend]
        cycle_tmin=tmin[:,:,istart:iend]
        cycle_tmax=tmax[:,:,istart:iend]
        # print('func 3')
        meanT_daily=0.5*(cycle_tmin+cycle_tmax)
        cycle_meanT=meanT_daily.mean(axis=2).astype('float16')
    else:
        cycle_meanT=np.zeros((aez.im_height,aez.im_width),dtype='float16')
        # cycle_meanT=np.empty((aez.im_height,aez.im_width),dtype='float16')
        # cycle_meanT[:]=np.float32(np.nan)
        # cycle_meanT[:]=0
    return cycle_meanT

In [ ]:
step_doy=1
istart_cycle_rain=np.arange(aez.doy_start-1,aez.doy_end,step_doy,dtype='int16')
cycle_len_rain=aez.cycle_len_rain.compute().astype('int16')
unique_cycles=np.unique(cycle_len_rain).astype('int16')

mask=np.where(aez.cycle_len_rain.compute()>0,1,0).astype('int8')
mask3D=np.broadcast_to(mask[:,:,np.newaxis],(aez.im_height,aez.im_width,aez.doy_end)).astype('int8')

minT_screen=np.where(mask3D,min_temp,0).astype('float16')
maxT_screen=np.where(mask3D,max_temp,0).astype('float16')
minT_daily_2year = np.tile(minT_screen, 2).compute()
maxT_daily_2year = np.tile(maxT_screen, 2).compute()
vardims=minT_daily_2year.shape

all_windows=np.empty((aez.im_height,aez.im_width,aez.doy_end),dtype='float16')

# mask.dtype,mask3D.dtype,minT_daily_2year.dtype


In [ ]:
istart_cycle_rain
aez.pet_daily

fig = plt.figure(figsize=(12,3))
plt.imshow(aez.pet_daily[:,:,0], cmap='viridis', vmin=0, vmax=0.5,interpolation='none')  
plt.colorbar(shrink=0.8)

In [ ]:
# unique_cycles=dask.delayed(unique_cycles)
minT_daily_2year=dask.delayed(minT_daily_2year)
maxT_daily_2year=dask.delayed(maxT_daily_2year)
# vardims=dask.delayed(vardims)

In [ ]:
# parallel, working but slow

# loop through 365 sliding growing season windows
for istart in istart_cycle_rain:
    task_list=[]
    print('window',istart+1,'of',len(istart_cycle_rain))
    # loop through growing season lengths 
    # (compute groups of pixels with the same growing season length together)
    # then combine into one map
    # task_list=[dask.delayed(calculateCycleQuantities)(istart,c_len,cycle_len_rain) for c_len in range(aez.doy_start,aez.doy_end+1)]
    # print('building task graph')
    for c_len in range(aez.doy_start-1,aez.doy_end+1):
        # print(c_len)
        # compute season variables
        task=dask.delayed(calculateCycleQuantities)(istart,c_len,cycle_len_rain,minT_daily_2year,maxT_daily_2year)
        task_list.append(task)
    # print('computing pixel values,',len(task_list),'tasks')
    results=dask.compute(*task_list)
    del task_list
    # print(results[0].shape)
    # aggregate pixel groups back together
    # print('stacking results')
    cycle_results=np.stack(results,axis=2)#,dtype='float32')
    # print(len(results),cycle_results.shape)
    # print('aggregating map')
    # print(cycle_results[300:310,50:60,4])
    cycle_meanT=np.nansum(cycle_results,axis=2)#,dtype='float32')
    # save seasonal variables for each window
    # print('saving window variables')
    all_windows[:,:,istart]=cycle_meanT

# all_windows.shape

In [ ]:
# working loops, not parallel
mask=np.where(cycle_len_rain>0,1,0)
results=[]
# loop through 365 sliding growing season windows
for istart in istart_cycle_rain[0:1]:
    # loop through growing season lengths 
    # (compute groups of pixels with the same growing season length together)
    # then combine into one map
    for c_len in range(aez.doy_start,aez.doy_end+1):
            # compute season variables
            result=calculateCycleQuantities(istart,c_len,cycle_len_rain)
            results.append(result)
            # print(result.shape)
            # aggregate pixels back together
    cycle_results=np.stack(results,axis=2,dtype='float32')
    cycle_meanT=np.nansum(cycle_results,axis=2)   
    # save seasonal variables for each window
    all_windows[:,:,istart]=cycle_meanT

all_windows.shape

In [ ]:
cycle_meanT=np.nansum(cycle_results,axis=2)
mask=np.where(cycle_len_rain>0,1,0)
cycle_meanT=np.where(mask,cycle_meanT,np.float32(np.nan))


npts=np.sum(np.where(np.isfinite(cycle_meanT),1,0))
print('npts=',npts)

fig = plt.figure(figsize=(12,3))
# plt.imshow(results[2][:,:,0], cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.imshow(cycle_meanT, cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.colorbar(shrink=0.8)
plt.show()

In [ ]:
cycle_results.shape,cycle_results

In [ ]:
test=np.where(~np.isfinite(cycle_results),0,cycle_results)
test
# cycle_meanT=np.nansum(cycle_results,axis=2)#,dtype='float32')
# # save seasonal variables for each window
# print('saving window variables')
# all_windows[:,:,istart]=cycle_meanT

In [ ]:
# working loops, not parallel

vardims=minT_daily_2year.shape

all_windows=np.empty((aez.im_height,aez.im_width,aez.doy_end),dtype='float32')
all_windows[:]=np.float32(np.nan)

cycle_meanT=np.empty((aez.im_height,aez.im_width),dtype='float32')
cycle_meanT[:]=np.float32(np.nan)
istart_cycle_rain=np.arange(aez.doy_start-1,aez.doy_end,step_doy,dtype=int)

# loop through 365 sliding growing season windows
for istart in istart_cycle_rain[0:2]:
    # loop through growing season lengths 
    # (compute groups of pixels with the same growing season length together)
    # then combine into one map
    for i,c_len in enumerate(unique_cycles):
        print(i,'of',len(unique_cycles))
        if (np.isfinite(c_len)&(c_len>0)):
            iend=istart+int(c_len)
            # find all pixels with this length of growing season
            cycle_mask=np.where(aez.cycle_len_rain.compute()==c_len,1,0)
            # expand dims
            cycle_mask3D=np.broadcast_to(cycle_mask[:,:,np.newaxis],vardims)
            # mask input variables in space
            tmin_masked=np.where(cycle_mask3D,minT_daily_2year,np.float32(np.nan))
            tmax_masked=np.where(cycle_mask3D,maxT_daily_2year,np.float32(np.nan))
            # subset input variables in time
            cycle_tmin=tmin_masked[:,:,istart:iend]
            cycle_tmax=tmax_masked[:,:,istart:iend]
            # compute season variables
            result=calculateCycleQuantities(cycle_tmin,cycle_tmax)
            # print(result.shape)
            # aggregate pixels back together
            cycle_meanT=np.where((np.isfinite(result))&(~np.isfinite(cycle_meanT)),result,cycle_meanT)
    # save seasonal variables for each window
    all_windows[:,:,istart]=cycle_meanT

all_windows.shape

In [ ]:
fig = plt.figure(figsize=(12,3))
# plt.imshow(results[2][:,:,0], cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.imshow(all_windows[:,:,1], cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.colorbar(shrink=0.8)
plt.show()

In [ ]:
# delay 3D minT_daily, maxT_daily, shortrad arrays

# chunk by cycle for calculateCycleQuantities, return 2D maps and stack in time
vardims=minT_daily_2year.shape

results=[]
for i,c_len in enumerate(unique_cycles[0:4]):
    if np.isfinite(c_len):
        # grids with this growing season length
        cycle_mask=np.where(aez.cycle_len_rain.compute()==c_len,1,0)
        cycle_mask3D=np.broadcast_to(cycle_mask[:,:,np.newaxis],vardims)
        tmin_masked=np.where(cycle_mask3D,minT_daily_2year,np.float32(np.nan))
        tmax_masked=np.where(cycle_mask3D,maxT_daily_2year,np.float32(np.nan))

        # 365 time windows of growing season length
        istart_cycle_rain=np.arange(aez.doy_start-1,aez.doy_end,step_doy,dtype=int)
        iend_cycle_rain=istart_cycle_rain+int(c_len)

        # if c_len != (iend_cycle_rain[i]-istart_cycle_rain[i]):
        #     print(c_len,(iend_cycle_rain[i]-istart_cycle_rain[i]))
        cycle_meanT=np.empty((aez.im_height,aez.im_width,aez.doy_end),dtype='float32')
        for j,(istart,iend) in enumerate(zip(istart_cycle_rain,iend_cycle_rain)):
            cycle_tmin=tmin_masked[:,:,istart:iend]
            cycle_tmax=tmax_masked[:,:,istart:iend]
            cycle_meanT[:,:,j]=calculateCycleQuantities(cycle_tmin,cycle_tmax)
            # print(istart,iend,cycle_tmin.shape)

        results.append(cycle_meanT)

# ###### THIS IS WHERE YOU LEFT OFF ###############    
#     fig = plt.figure(figsize=(12,3))
#     plt.imshow(cycle_mask, cmap='viridis', vmin=0, vmax=1,interpolation='none')  
#     plt.colorbar(shrink=0.8)
#     plt.show()


    results

In [ ]:
agg_result=np.empty((aez.im_height,aez.im_width,aez.doy_end),dtype='float32')
agg_result[:]=np.float32(np.nan)

for i,r in enumerate(results):
    print(i)
    agg_result=np.where((np.isfinite(r))&~np.isfinite(agg_result),r,agg_result)

In [ ]:
len(results), results[2]

fig = plt.figure(figsize=(12,3))
# plt.imshow(results[2][:,:,0], cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.imshow(results[2][:,:,0], cmap='viridis', vmin=-20, vmax=20,interpolation='none')  
plt.colorbar(shrink=0.8)
plt.show()


In [ ]:
# CropSimulation.py
# create arrays with all the possible season start doys and end doys to replace looping
start_doy=aez.doy_start
end_doy=aez.doy_end
step_doy=1
start_cycle_rain=np.arange(start_doy-1,end_doy,step_doy,dtype='float32')
start_cycle_rain=np.broadcast_to(start_cycle_rain[np.newaxis,np.newaxis,:],(aez.im_height,aez.im_width,aez.doy_end))
# start_cycle_rain=np.where(multi_screen3D.compute(),start_cycle_rain,np.float32(np.nan))
# # subtracting 1 in the original code cuts off a day of the season, fixing here. proof in the commentd line (all -1)
# #  np.unique(end_cycle_rain-start_cycle_rain-np.broadcast_to(aez.cycle_len_rain[:,:,np.newaxis],(aez.im_height,aez.im_width,end_doy))).compute()
end_cycle_rain=start_cycle_rain+np.broadcast_to(aez.cycle_len_rain.compute()[:,:,np.newaxis],(aez.im_height,aez.im_width,aez.doy_end))
# # start_cycle_rain,end_cycle_rain
start_cycle_rain.shape, end_cycle_rain.shape

# start_cycle_rain=np.arange(start_doy-1,end_doy,step_doy,dtype='float32')
# end_cycle_rain=aez.cycle_len_rain.compute()+start_cycle_rain
# end_cycle_rain

In [ ]:
test=meanT_daily(300,50,)

# cycle_meanT=np.where

# meanT_mean = np.mean( meanT_daily );
# dT_mean = np.mean( dT_daily );
# Rg = np.mean( shortRad_daily );

In [ ]:
# BioMassCalc.setClimData
# compute dT_daily

dT_daily=np.empty((aez.im_height,aez.im_width,aez.doy_end*2),dtype='float32')
dT_daily[:]=np.float32(np.nan)


dT_daily[:,:,0:31]    = 0.0278 + 0.3301*minT_daily_2year[:,:,0:31]    + 0.6716*maxT_daily_2year[:,:,0:31]
dT_daily[:,:,31:59]   = 0.078  + 0.3345*minT_daily_2year[:,:,31:59]   + 0.6672*maxT_daily_2year[:,:,31:59]
dT_daily[:,:,59:90]   = 0.0770 + 0.3392*minT_daily_2year[:,:,59:90]   + 0.6642*maxT_daily_2year[:,:,59:90]
dT_daily[:,:,90:120]  = 0.2276 + 0.3466*minT_daily_2year[:,:,90:120]  + 0.6536*maxT_daily_2year[:,:,90:120]
dT_daily[:,:,120:151] = 0.2494 + 0.3399*minT_daily_2year[:,:,120:151] + 0.6576*maxT_daily_2year[:,:,120:151]
dT_daily[:,:,151:181] = 0.9955 + 0.3335*minT_daily_2year[:,:,151:181] + 0.6628*maxT_daily_2year[:,:,151:181]
dT_daily[:,:,181:212] = 0.2624 + 0.3351*minT_daily_2year[:,:,181:212] + 0.659*maxT_daily_2year[:,:,181:212]
dT_daily[:,:,212:243] = 0.2860 + 0.3297*minT_daily_2year[:,:,212:243] + 0.6612*maxT_daily_2year[:,:,212:243]
dT_daily[:,:,243:273] = 0.3492 + 0.3410*minT_daily_2year[:,:,243:273] + 0.6515*maxT_daily_2year[:,:,243:273]
dT_daily[:,:,273:304] = 0.1598 + 0.3394*minT_daily_2year[:,:,273:304] + 0.6591*maxT_daily_2year[:,:,273:304]
dT_daily[:,:,304:334] = 0.1156 + 0.3476*minT_daily_2year[:,:,304:334] + 0.6571*maxT_daily_2year[:,:,304:334]
# CAN THIS BE CORRECT? THE WHOLE 2nd YEAR VALUES ARE ASSIGNED AS DECEMBER IN ORIGINAL CODE
dT_daily[:,:,334:]    = -0.0617+ 0.3462*minT_daily_2year[:,:,334:]    + 0.6649*maxT_daily_2year[:,:,334:]

# dT_daily


In [ ]:
# BioMassCalc.__init__
# get the latitude band number for radiation calculations
lat = np.abs(aez.latitude)
lat_index1 = np.floor(np.abs(aez.latitude)/10)
lat_index2 = np.ceil(np.abs(aez.latitude)/10)
lat[50,50].compute(),lat_index1[50,50].compute(),lat_index2[50,50].compute()
lat_index1.shape

In [ ]:
'''Max Radiation''' # Ac shape = (lat band number, month) 
Ac = np.array([[343,360,369,364,349,337,342,357,368,365,349,337],# correct
    [299,332,359,375,377,374,375,377,369,345,311,291],# correct
    [249,293,337,375,394,400,399,386,357,313,264,238], #correct
    [191,245,303,363,400,417,411,384,333,270,210,179], # correct
    [131,190,260,339,396,422,413,369,298,220,151,118], # correct
        [73,131,207,304,380,418,405,344,254,163, 92, 61], # correct
        [22, 72,149,260,356,408,389,309,201,103, 37, 14], # correct
        [0, 20, 89,209,331,408,380,269,142, 45,  2,  0], # correct
        [0,  0, 28,162,334,424,393,248, 81,  3,  0,  0], # correct
        [0,  0,  0,154,339,428,397,252, 40,  0,  0,  0]]); # correct

'''Biomass in open day'''
bc = np.array([[413,424,429,426,417,410,413,422,429,427,418,410], # same in de Wit (1965), but different in Fortran routine
    [376,401,422,437,440,440,440,439,431,411,385,370], # correct
    [334,371,407,439,460,468,465,451,425,387,348,325], # correct
    [281,333,385,437,471,489,483,456,412,356,299,269], # correct
    [218,283,353,427,480,506,497,455,390,314,241,204], # same in de Wit (slightly different in FORTRAN routine )
    [147,223,310,409,484,522,509,448,358,260,173,130], # one value different, now remodified.
        [66,151,254,383,487,544,523,436,316,195, 94, 49], # correct
        [0, 65,185,350,506,612,575,427,262,114,  7,  0], # correct
        [0,  0, 94,333,571,663,632,474,195, 11,  0,  0],# correct
        [0,  0,  0,371,588, 677,646,497,167,  0,  0,  0]]); # same as de Wit (some big difference in FORTRAN)

''' Biomass in cloudy day '''
bo = np.array([[219,226,230,228,221,216,218,225,230,228,222,216], # same as de Wit (difference of 1 in FORTRAN)
    [197,212,225,234,236,235,236,235,230,218,203,193], # correct
    [170,193,215,235,246,250,249,242,226,203,178,164], # correct
    [137,168,200,232,251,261,258,243,216,182,148,130], # correct
        [99,137,178,223,253,268,263,239,200,155,112, 91], # correct
        [60,100,150,207,251,273,265,230,178,121, 73, 51], # correct
        [19, 60,114,187,245,276,265,216,148, 82, 31, 11], # correct
        [0, 16, 74,158,241,291,273,200,112, 38,  1,  0], # correct
        [0,  0, 24,133,257,318,297,196, 69,  2,  0,  0], # correct
        [0,  0,  0,131,269,319,302,215, 35,  0,  0,  0]]); # correct

'''Pm values table''' # there's a big look-up table references for PM look-up table, for maize, it's correct.
PmIndexExtDtTemp = np.array([5.0,10.0,15.0,20.0,25.0,30.0,35.0]);
PmIndexExt = np.array([[0.0,15.0,20.0,20.0,  15.0, 5.0, 0.0],
    [0.0, 0.0,15.0,32.5,35.0,35.0,35.0],
    [0.0, 0.0, 5.0,45.0,65.0,65.0,65.0],
    [0.0, 5.0,45.0,65.0, 65.0,65.0,65.0]]);

doy_middle_of_month = np.arange(0,12)*30 + 15 # Calculate doy of middle of month
doy_middle_of_month



In [ ]:
tlat=300
tlon=50
tday=300
start_cycle_rain[tlat,tlon,tday],end_cycle_rain[tlat,tlon,tday],lat_index1[tlat,tlon].compute()
# np.arange(start_cycle_rain[300,50,0],end_cycle_rain[300,50,0]+1)
x=np.arange(start_cycle_rain[tlat,tlon,tday],end_cycle_rain[tlat,tlon,tday])
lat[tlat,tlon]

In [ ]:
tlat=300
tlon=50
tday=0

x_out=np.arange(start_cycle_rain[tlat,tlon,tday],end_cycle_rain[tlat,tlon,tday])
x_in=doy_middle_of_month
y_in1=Ac[int(lat_index1[tlat,tlon].compute()),:]
y_in2=Ac[int(lat_index2[tlat,tlon].compute()),:]

Ac_interp1 = np.interp(x_out, x_in, y_in1)
Ac_interp2 = np.interp(x_out, x_in, y_in2)
Ac_interp = Ac_interp1 + (int(lat[tlat,tlon].compute())-int(lat_index1[tlat,tlon].compute())*10)*((Ac_interp2-Ac_interp1)/10)

Ac_mean = np.mean( Ac_interp )

plt.plot(x_out,Ac_interp)



In [ ]:
aez.cycle_len_rain[tlat,tlon].compute()


In [ ]:
''' Calculate average values of Ac, bc, bo, meanT, dT  in the season '''
Ac_interp1 = np.interp(np.arange(cycle_begin, cycle_end+1), doy_middle_of_month, Ac[int(lat_index1),:])
Ac_interp2 = np.interp(np.arange(cycle_begin, cycle_end+1), doy_middle_of_month, Ac[int(lat_index2),:])
Ac_interp = Ac_interp1 + (int(lat)-int(lat_index1)*10)*((Ac_interp2-Ac_interp1)/10)
Ac_mean = np.mean( Ac_interp );


In [ ]:
def simulateCropCycle(self, start_doy=1, end_doy=365, step_doy=1, leap_year=False):
    """Running the crop cycle calculation/simulation

    Args:
        start_doy (int, optional): Starting Julian day for simulating period. Defaults to 1.
        end_doy (int, optional): Ending Julian day for simulating period. Defaults to 365.
        step_doy (int, optional): Spacing (in days) between 2 adjacent crop simulations. Defaults to 1.
        leap_year (bool, optional): whether or not the simulating year is a leap year. Defaults to False.

    """        

    # just a counter to keep track of progress
    count_pixel_completed = 0
    total = self.im_height * self.im_width
    # this stores final result
    self.final_yield_rainfed = np.zeros((self.im_height, self.im_width))
    self.final_yield_irrig = np.zeros((self.im_height, self.im_width))
    self.crop_calender_irr = np.zeros(
        (self.im_height, self.im_width), dtype=int)
    self.crop_calender_rain = np.zeros(
        (self.im_height, self.im_width), dtype=int)
    
    if not self.perennial:
        self.fc2 = np.zeros((self.im_height, self.im_width))

    
    self.fc1_rain = np.zeros((self.im_height, self.im_width))
    self.fc1_irr = np.zeros((self.im_height, self.im_width))


    for i_row in range(self.im_height):

        for i_col in range(self.im_width):

            print('\nrow_col= {}_{}'.format(i_row, i_col))

            

            # check current location (pixel) is outside of study area or not. if it's outside of study area goes to next location (pixel)
            # Those unsuitable
            if self.set_mask:
                if self.im_mask[i_row, i_col] == self.nodata_val:
                    count_pixel_completed = count_pixel_completed + 1
                    print('\rDone %: ' + str(round(count_pixel_completed /
                    total*100, 2)), end='\r')
                    continue

            # 2. Permafrost screening
            if self.set_Permafrost_screening:
                if np.logical_or(self.permafrost_class[i_row, i_col] == 1, self.permafrost_class[i_row, i_col] == 2):
                    count_pixel_completed = count_pixel_completed + 1
                    print('\rDone %: ' + str(round(count_pixel_completed /
                    total*100, 2)), end='\r')
                    continue

            # Thermal Climate Screening
            if self.set_tclimate_screening:
                if self.t_climate[i_row, i_col] in self.no_t_climate:
                    count_pixel_completed = count_pixel_completed + 1
                    
                    print('\rDone %: ' + str(round(count_pixel_completed /
                    total*100, 2)), end='\r')
                    
                    continue
            
            # Minimum temperature requirement Checking
            if np.round(np.mean(self.meanT_daily[i_row, i_col,:]), 0) < self.min_temp:
                count_pixel_completed = count_pixel_completed + 1
                    
                print('\rDone %: ' + str(round(count_pixel_completed /
                    total*100, 2)), end='\r')
                continue
            # print(r'\nRow{}, Col{} '.format(i_row, i_col), end = '\n')

            count_pixel_completed = count_pixel_completed + 1       
            # this allows handing leap and non-leap year differently. This is only relevant for monthly data because this value will be used in interpolations.
            # In case of daily data, length of vector will be taken as number of days in  a year.
            if leap_year:
                days_in_year = 366
            else:
                days_in_year = 365

            # extract climate data for particular location. And if climate data are monthly data, they are interpolated as daily data
            if self.set_monthly:
                obj_utilities = UtilitiesCalc.UtilitiesCalc()

                minT_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.minT_monthly[i_row, i_col, :], 1, days_in_year)
                maxT_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.maxT_monthly[i_row, i_col, :], 1, days_in_year)
                shortRad_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.shortRad_monthly[i_row, i_col, :],  1, days_in_year, no_minus_values=True)
                wind2m_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.wind2m_monthly[i_row, i_col, :],  1, days_in_year, no_minus_values=True)
                totalPrec_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.totalPrec_monthly[i_row, i_col, :],  1, days_in_year, no_minus_values=True)
                rel_humidity_daily_point = obj_utilities.interpMonthlyToDaily(
                    self.rel_humidity_monthly[i_row, i_col, :],  1, days_in_year, no_minus_values=True)
            else:
                minT_daily_point = self.minT_daily[i_row, i_col, :]
                maxT_daily_point = self.maxT_daily[i_row, i_col, :]
                shortRad_daily_point = self.shortRad_daily[i_row, i_col, :]
                wind2m_daily_point = self.wind2m_daily[i_row, i_col, :]
                totalPrec_daily_point = self.totalPrec_daily[i_row, i_col, :]
                rel_humidity_daily_point = self.rel_humidity_daily[i_row, i_col, :]
                

            # calculate ETO for full year for particular location (pixel) 7#
            obj_eto = ETOCalc.ETOCalc(
                1, minT_daily_point.shape[0], self.latitude[i_row, i_col], self.elevation[i_row, i_col])

            # 7. New Modification
            shortRad_daily_point_MJm2day = (shortRad_daily_point*3600*24)/1000000 # convert w/m2 to MJ/m2/day (Correct)

            # 7. Minor change for validation purposes: shortRad_daily_point is replaced in shortRad_dailyy_point_MJm2day. (Sunshine hour data for KB Etocalc)
            obj_eto.setClimateData(minT_daily_point, maxT_daily_point,
                                    wind2m_daily_point, shortRad_daily_point_MJm2day, rel_humidity_daily_point)
            pet_daily_point = obj_eto.calculateETO()

            

            """No adjustment of cycle length, LAI and HI for non-perennials"""
            if not self.perennial:
                
                self.cycle_len_rain = self.cycle_len
                self.LAi_rain = self.LAi
                self.HI_rain = self.HI

                self.cycle_len_irr = self.cycle_len
                self.LAi_irr = self.LAi
                self.HI_irr = self.HI
            
            else:
                """Adjustment of cycle length, LAI and HI for Perennials"""
                self.set_adjustment = True

                """ Adjustment for RAINFED conditions"""

                if self.LGP[i_row, i_col] < self.cycle_len:

                    # LGP duration will be efficient cycle length for rainfed conditions
                    # Later, we use LGP length to adjust for LAI and HI for rainfed conditions
                    self.cycle_len_rain = int(self.LGP[i_row, i_col])
                    self.adjustForPerennialCrop(
                        self.cycle_len_rain, aLAI=self.aLAI, bLAI=self.bLAI, aHI=self.aHI, bHI=self.bHI, rain_or_irr='rain')
                else:
                    self.cycle_len_rain = self.cycle_len
                    self.LAi_rain = self.LAi
                    self.HI_rain = self.HI
            
                """ Adjustment for IRRIGATED conditions"""

                """Use LGPT5 for minimum temperature less than or equal to five deg Celsius"""
                if self.min_temp <= 5:

                    if self.LGPT5[i_row, i_col] < self.cycle_len:

                        self.cycle_len_irr = int(self.LGPT5[i_row, i_col].copy())
                        self.adjustForPerennialCrop(
                            self.cycle_len_irr, aLAI=self.aLAI, bLAI=self.bLAI, aHI=self.aHI, bHI=self.bHI, rain_or_irr='irr')

                    else:
                        self.cycle_len_irr = self.cycle_len
                        self.LAi_irr = self.LAi
                        self.HI_irr = self.HI

                """Use LGPT10 for minimum temperature greater than five deg Celsius"""

                if self.min_temp > 5:

                    if self.LGPT10[i_row, i_col] < self.cycle_len:

                        self.cycle_len_irr = int(self.LGPT10[i_row, i_col].copy())
                        self.adjustForPerennialCrop(
                            self.cycle_len_irr, aLAI=self.aLAI, bLAI=self.bLAI, aHI=self.aHI, bHI=self.bHI, rain_or_irr='irr')

                    else:
                        self.cycle_len_irr = self.cycle_len
                        self.LAi_irr = self.LAi
                        self.HI_irr = self.HI
            

            # Empty arrays that stores yield estimations and fc1 and fc2 of all cycles per particular location (pixel)
            yield_of_all_crop_cycles_rainfed = np.empty(0, dtype= np.float16)
            yield_of_all_crop_cycles_irrig = np.empty(0, dtype= np.float16)

            fc1_rain_lst = np.empty(0, dtype= np.float16)
            fc1_irr_lst = np.empty(0, dtype= np.float16)

            fc2_lst = np.empty(0, dtype= np.float16)


            """ Calculation of each individual day's yield for rainfed and irrigated conditions"""

            for i_cycle in range(start_doy-1, end_doy, step_doy):

                """Repeat the climate data two times and concatenate for computational convenience. If perennial, the cycle length
                will be different for separate conditions"""
                print('Cycle No.{}'.format(i_cycle), end = '\n')

                minT_daily_2year = np.tile(minT_daily_point, 2)
                maxT_daily_2year = np.tile(maxT_daily_point, 2)
                shortRad_daily_2year = np.tile(shortRad_daily_point, 2)
                
                totalPrec_daily_2year = np.tile(totalPrec_daily_point, 2)
                pet_daily_2year = np.tile(pet_daily_point, 2)

                # print('Tiling complete')

                

                """ Time slicing tiled climate data with corresponding cycle lengths for rainfed and irrigated conditions"""
                """For rainfed"""

                # extract climate data within the season to pass in to calculation classes
                minT_daily_season_rain = minT_daily_2year[i_cycle: i_cycle +
                                                            int(self.cycle_len_rain)-1]
                maxT_daily_season_rain = maxT_daily_2year[i_cycle: i_cycle +
                                                            int(self.cycle_len_rain)-1]
                shortRad_daily_season_rain = shortRad_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_rain)-1]
                pet_daily_season_rain = pet_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_rain)-1]
                totalPrec_daily_season_rain = totalPrec_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_rain)-1]
                
                """For irrigated"""
                # extract climate data within the season to pass in to calculation classes
                minT_daily_season_irr = minT_daily_2year[i_cycle: i_cycle +
                                                            int(self.cycle_len_irr)-1]
                maxT_daily_season_irr = maxT_daily_2year[i_cycle: i_cycle +
                                                            int(self.cycle_len_irr)-1]
                shortRad_daily_season_irr = shortRad_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_irr)-1]
                pet_daily_season_irr = pet_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_irr)-1]
                totalPrec_daily_season_irr = totalPrec_daily_2year[
                    i_cycle: i_cycle+int(self.cycle_len_irr)-1]
                
                # print('Climate time slicing complete')



                """Thermal Screening using each cycle length for rainfed and irrigated conditions"""

                """ For the perennial, the adjusted cycle length for rainfed and irrigated conditions will be used. For the rest,
                    the user-specified cycle length will be applied"""

                """Creating Thermal Screening object classes for perennial rainfed and irrigated conditions"""
                obj_screening_rain = ThermalScreening.ThermalScreening()
                obj_screening_irr = ThermalScreening.ThermalScreening()


                obj_screening_rain.setClimateData(
                    minT_daily_season_rain, maxT_daily_season_rain)
                obj_screening_irr.setClimateData(
                    minT_daily_season_irr, maxT_daily_season_irr)


                if self.set_lgpt_screening:
                    obj_screening_rain.setLGPTScreening(
                        no_lgpt=self.no_lgpt, optm_lgpt=self.optm_lgpt)
                    obj_screening_irr.setLGPTScreening(
                        no_lgpt=self.no_lgpt, optm_lgpt=self.optm_lgpt)

                # 5 Modification (SWH)
                if self.set_Tsum_screening:
                    obj_screening_rain.setTSumScreening(
                        LnS=self.LnS, LsO=self.LsO, LO=self.LO, HnS=self.HnS, HsO=self.HsO, HO=self.HO)
                    obj_screening_irr.setTSumScreening(
                        LnS=self.LnS, LsO=self.LsO, LO=self.LO, HnS=self.HnS, HsO=self.HsO, HO=self.HO)

                # 8 Modification
                if self.setTypeBConstraint:

                    if self.perennial:
                        obj_screening_rain.applyTypeBConstraint(
                            data=self.data, input_temp_profile=obj_screening_rain.tprofile, perennial_flag=True)
                        obj_screening_irr.applyTypeBConstraint(
                            data=self.data, input_temp_profile=obj_screening_irr.tprofile, perennial_flag=True)
                    else:
                        obj_screening_rain.applyTypeBConstraint(
                        data=self.data, input_temp_profile=obj_screening_rain.tprofile, perennial_flag=False)
                        obj_screening_irr.applyTypeBConstraint(
                        data=self.data, input_temp_profile=obj_screening_irr.tprofile, perennial_flag=False)

                fc1_rain = 1.
                fc1_irr = 1.
                # print('Original fc1_rain =', fc1_rain)
                # print('Original fc1_irr =', fc1_irr)

                fc1_rain = obj_screening_rain.getReductionFactor2()  # fc1 for rainfed condition
                fc1_irr = obj_screening_irr.getReductionFactor2()  # fc1 for irrigated condition

            

                # if not obj_screening_rain.getSuitability():
                #     continue

                # else:
                #     fc1_rain = obj_screening_rain.getReductionFactor2()  # fc1 for rainfed condition

                # if not obj_screening_irr.getSuitability():
                #     continue
                # else:
                #     fc1_irr = obj_screening_irr.getReductionFactor2()  # fc1 for irrigated condition
                
                if fc1_rain == None or fc1_irr == None:
                    raise Exception('Fc1 not returned in Thermal Screening Calculation. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))
                
                if fc1_rain == np.nan or fc1_irr == np.nan:
                    raise Exception('Fc1 nan value returned. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))
                
                # Appending individual cycle's fc1 for rainfed and irrigated condition
                # print('After fc1 irr=', fc1_irr)
                # print('After fc1 rain=', fc1_rain)

                # print('Thermal Screening calculation complete')
                
                fc1_irr_lst = np.append(fc1_irr_lst, fc1_irr)
                fc1_rain_lst = np.append(fc1_rain_lst, fc1_rain)


                # print('fc1_irr_lst = ', fc1_irr_lst)
                # print('fc1_rain_lst = ', fc1_rain_lst)

                if len(fc1_irr_lst) != i_cycle+1:
                    print('fc1_irr_lst = ', fc1_irr_lst)
                    raise Exception('Fc1 irr not properly appended. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))
                
                elif len(fc1_rain_lst)!= i_cycle+1:
                    raise Exception('Fc1 rain not properly appended. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))


                """Biomass Calculation relevant to perennials and non-perennials for IRRIGATED conditions"""
                """IRRIGATED"""
                obj_maxyield_irr = BioMassCalc.BioMassCalc(
                    i_cycle+1, i_cycle+1+self.cycle_len_irr-1, self.latitude[i_row, i_col])
                obj_maxyield_irr.setClimateData(
                    minT_daily_season_irr, maxT_daily_season_irr, shortRad_daily_season_irr)
                obj_maxyield_irr.setCropParameters(
                    self.LAi_irr, self.HI_irr, self.legume, self.adaptability)
                obj_maxyield_irr.calculateBioMass()
                est_yield_irrigated = obj_maxyield_irr.calculateYield()

                # reduce thermal screening factor
                est_yield_irrigated = est_yield_irrigated * fc1_irr

                if est_yield_irrigated == None or est_yield_irrigated == np.nan:
                    raise Exception('Irrigated Yield not returned. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))


                """append current cycle yield to a list IRRIGATED"""
                yield_of_all_crop_cycles_irrig = np.append(yield_of_all_crop_cycles_irrig, est_yield_irrigated)

                if len(yield_of_all_crop_cycles_irrig) != i_cycle+1:
                    raise Exception('Irr Yield cycles not appended. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))

                



                """A separate biomass calculation RAINFED for Non-Perennials"""
                obj_maxyield_rain = BioMassCalc.BioMassCalc(
                    i_cycle+1, i_cycle+1+self.cycle_len_rain-1, self.latitude[i_row, i_col])
                obj_maxyield_rain.setClimateData(
                    minT_daily_season_rain, maxT_daily_season_rain, shortRad_daily_season_rain)
                obj_maxyield_rain.setCropParameters(
                    self.LAi_rain, self.HI_rain, self.legume, self.adaptability)
                obj_maxyield_rain.calculateBioMass()
                est_yield_rainfed = obj_maxyield_rain.calculateYield()

                # reduce thermal screening factor
                est_yield_rainfed = est_yield_rainfed * fc1_rain

                if est_yield_rainfed == None or est_yield_rainfed == np.nan:
                    raise Exception('Biomass Yield for rainfed not returned. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))



                """For RAINFED Perennials, fc2 will be zero (No Need for Crop Water Requirement calculation)"""
                if self.perennial:
                    est_yield_rainfed = est_yield_rainfed * 1

                    """append current cycle yield to a list RAINFED"""
                    yield_of_all_crop_cycles_rainfed = np.append(yield_of_all_crop_cycles_rainfed, est_yield_rainfed)

                else:
                    obj_cropwat = CropWatCalc.CropWatCalc(
                        i_cycle+1, i_cycle+1+self.cycle_len_rain-1)
                    obj_cropwat.setClimateData(
                        pet_daily_season_rain, totalPrec_daily_season_rain)
                    
                    # check Sa is a raster or single value and extract Sa value accordingly
                    if len(np.array(self.Sa).shape) == 2:
                        Sa_temp = self.Sa[i_row, i_col]
                    else:
                        Sa_temp = self.Sa
                    obj_cropwat.setCropParameters(self.d_per, self.kc, self.kc_all, self.yloss_f,
                                                    self.yloss_f_all, est_yield_rainfed, self.D1, self.D2, Sa_temp, self.pc)
                    est_yield_moisture_limited = obj_cropwat.calculateMoistureLimitedYield()

                    if est_yield_moisture_limited == None or est_yield_moisture_limited == np.nan:
                        raise Exception('Crop Water Yield not returned. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))

                    fc2_value = obj_cropwat.getfc2factormap()

                    if fc2_value == None or fc2_value == np.nan:
                        raise Exception('fc2 value not returned. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))

                    """append current cycle yield to a list RAINFED"""
                    yield_of_all_crop_cycles_rainfed = np.append(yield_of_all_crop_cycles_rainfed, est_yield_moisture_limited)
                    fc2_lst = np.append(fc2_lst, fc2_value)

                    if len(yield_of_all_crop_cycles_rainfed) != i_cycle+1:
                        raise Exception('Rainfed yield list not properly appended')
                    elif len(fc2_lst) != i_cycle+1:
                        raise Exception('Fc2 list not appended properly. Row_{}_col_{}_Cycle_{}'.format(i_row, i_col, i_cycle))

                        # print(r'rain_yield_length = ', len(yield_of_all_crop_cycles_rainfed))
                        # print(r'fc2 = ', fc2_value)
                        # print(r'fc2 list =', len(fc2_lst))


            """Getting Maximum Attainable Yield from the list for irrigated and rainfed conditions and the Crop Calendar"""

            # get agro-climatic yield and crop calendar for IRRIGATED condition
            if np.logical_and(len(yield_of_all_crop_cycles_irrig) == len(fc1_irr_lst), len(yield_of_all_crop_cycles_irrig) == len(fc1_irr_lst)):

                self.final_yield_irrig[i_row, i_col] = np.max(yield_of_all_crop_cycles_irrig) # Maximum attainable yield

                i = np.where(yield_of_all_crop_cycles_irrig == np.max(yield_of_all_crop_cycles_irrig))[0][0] # index of maximum yield

                self.crop_calender_irr[i_row, i_col] = int(i+1)*step_doy # Crop calendar for irrigated condition

                self.fc1_irr[i_row, i_col] = fc1_irr_lst[i] # fc1 irrigated for the specific crop calendar DOY

            # get agro-climatic yield and crop calendar for RAINFED condition
            if np.logical_and(len(yield_of_all_crop_cycles_rainfed) == len(fc1_rain_lst), len(yield_of_all_crop_cycles_rainfed) == len(fc1_rain_lst)):
                self.final_yield_rainfed[i_row, i_col] = np.max(yield_of_all_crop_cycles_rainfed) # Maximum attainable yield

                i1 = np.where(yield_of_all_crop_cycles_rainfed == np.max(yield_of_all_crop_cycles_rainfed))[0][0] # index of maximum yield
                
                self.crop_calender_rain[i_row, i_col] = int(i1+1) * step_doy # Crop calendar for rainfed condition
                
                self.fc1_rain[i_row, i_col] = fc1_rain_lst[i1]
                
                if not self.perennial:
                    self.fc2[i_row, i_col] = fc2_lst[i1]


            print('\rDone %: ' + str(round(count_pixel_completed / total*100, 2)), end='\r')
    
    print('\nSimulations Completed !')

### Maximum Attainable Yield for Rainfed and Irrigated Condition/ Optimum starting date


Yield Classification
1.   not suitable (yields between 0% and 20%)
2.   marginally suitable (yields between 20% and 40%)
3.   moderately suitable (yields between 40% and 60%)
4. suitable (yields between 60% and 80%)
5. very suitable (yields are equivalent to 80% or more of the overall maximum yield)



In [ ]:
# Now, showing the estimated and highly obtainable yield of a particular crop, results are in kg / hectare
yield_map_rain = aez.getEstimatedYieldRainfed()  # for rainfed
yield_map_irr = aez.getEstimatedYieldIrrigated()  # for irrigated

# Optimum cycle start date, the date when the highest yield are produced referenced from the start of crop cycle
starting_date_rain = aez.getOptimumCycleStartDateRainfed()
starting_date_irr = aez.getOptimumCycleStartDateIrrigated()

## get classified output of yield
# yield_map_rain_class = obj_util.classifyFinalYield(yield_map_rain)
# yield_map_irr_class = obj_util.classifyFinalYield(yield_map_irr)


In [ ]:
'''visualize result'''

"""Yield Maps"""
plt.imshow(yield_map_rain, vmax = np.max([yield_map_irr, yield_map_rain]), vmin = 0)
plt.colorbar()
plt.title('Rainfed Yield')
plt.show()

plt.imshow(yield_map_irr, vmax = np.max([yield_map_irr, yield_map_rain]), vmin = 0)
plt.colorbar()
plt.title('Irrigated Yield')
plt.show()

"""Starting Date (Crop Calendar)"""
plt.imshow(starting_date_rain, vmin= 0, vmax = 366)
plt.colorbar()
plt.title('Starting Date Rainfed')
plt.show()

plt.imshow(starting_date_irr, vmin= 0, vmax = 366)
plt.colorbar()
plt.title('Starting Date Irrigated')
plt.show()

"""Classified Yield"""
# plt.imshow(yield_map_rain_class)
# plt.colorbar()
# plt.title('Classified Rainfed Yield')
# plt.show()
# plt.imshow(yield_map_irr_class)
# plt.colorbar()
# plt.show('Classified Irrigated Yield')
# plt.show()

In [ ]:
# '''save result'''

obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_yield_rain.tif', yield_map_rain)
obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_yield_irr.tif', yield_map_irr)

obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_starting_date_rain.tif', starting_date_rain)
obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_starting_date_irr.tif', starting_date_irr)

# obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\CropSuitability_rain_class.tif',yield_map_rain_class)
# obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\CropSuitability_irr_class.tif',yield_map_irr_class)

In [ ]:
""" New outputs from crop simulation: Moisture reduction factor (fc2) """

"""
Perennials are considered no limitation due to moisture deficit/ This function is exclusive to non-perennials.
No need to run this cell for perennials. 
"""

if is_perennial:
    pass
else:
    fc2 = aez.getMoistureReductionFactor()
    """visualizing the result"""
    plt.imshow(fc2, vmin= 0, vmax = 1)
    plt.colorbar()
    plt.title('fc2 (Moisture limited reduction factor)')
    plt.show()


In [ ]:
# New outputs from crop simulation: Thermal reduction factor (fc1)

# Perennial will provide a python list of 2D np arrays as [fc1_rainfed, fc2_irrigated]
fc1 = aez.getThermalReductionFactor()

if type(fc1) == list:
    fc1_rain = fc1[0] # fc1 for rainfed conditions
    fc1_irr = fc1[1] # fc1 for irrigated conditions

    """visualizing the result"""
    plt.imshow(fc1_rain, vmin= 0, vmax = 1)
    plt.colorbar()
    plt.title('Thermal reduction factor rainfed')
    plt.show()

    """visualizing the result"""
    plt.imshow(fc1_irr, vmin= 0, vmax = 1)
    plt.colorbar()
    plt.title('Thermal reduction factor irrigated')
    plt.show()
else:
    """visualizing the result"""
    plt.imshow(fc1, vmin= 0, vmax = 1)
    plt.colorbar()
    plt.title('Thermal reduction factor (both conditions)')
    plt.show()

In [ ]:
"""Saving the thermal reduction factor as rasters"""

if type(fc1)==list:
    obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_fc1_rain.tif',fc1[0])
    obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_fc1_irr.tif',fc1[1])
else:
    obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_fc1.tif',fc1)



In [ ]:
"""Saving the moisture reduction factor as raster. Skip this for perennials"""
if is_perennial:
    pass
else:
    obj_util.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB2\sugar_fc2.tif',fc2)

<hr>

### END OF MODULE 2: CROP SIMULATION

<hr>